<a href="https://colab.research.google.com/github/sanjana251/ML-ACTIVITY/blob/main/NB_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import pandas as pd

In [4]:
#Load Dataset
df = pd.read_csv('PlayTennis_NB_Example.csv')

# Display the first 5 rows of the dataset
print(df.head())


  Day   Outlook Temperature Humidity    Wind PlayTennis
0  D1     Sunny         Hot     High    Weak         No
1  D2     Sunny         Hot     High  Strong         No
2  D3  Overcast         Hot     High    Weak        Yes
3  D4      Rain        Mild     High    Weak        Yes
4  D5      Rain        Cool   Normal    Weak        Yes


In [5]:
# Test Case is classsify the follwoing instance <Outlook = Sunny, Temperture = Cool, Humidity = High, Wind = Strong>
test_case = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}

# Calculate the prior probabilities for each class label
prior_probabilities = df['PlayTennis'].value_counts(normalize=True).to_dict()

# Round the prior probabilities to 2 decimal places
prior_probabilities = {k: round(v, 2) for k, v in prior_probabilities.items()}
print("Rounded Prior Probabilities: ", prior_probabilities)# Test Case is classsify the follwoing instance <Outlook = Sunny, Temperture = Cool, Humidity = High, Wind = Strong>
test_case = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}

# Calculate the prior probabilities for each class label
prior_probabilities = df['PlayTennis'].value_counts(normalize=True).to_dict()

# Round the prior probabilities to 2 decimal places
prior_probabilities = {k: round(v, 2) for k, v in prior_probabilities.items()}
print("Rounded Prior Probabilities: ", prior_probabilities)

Rounded Prior Probabilities:  {'Yes': 0.64, 'No': 0.36}
Rounded Prior Probabilities:  {'Yes': 0.64, 'No': 0.36}


In [6]:
# Compute the likelihood probabilities for each feature given the class label
likelihood_probabilities = {}

# Exclude the target variable and the Day columen from the features
for feature in df.columns[1:-1]:
    likelihood_probabilities[feature] = {}
    for class_label in df['PlayTennis'].unique():
        # Calculate the likelihood probability for each feature given the class label
        likelihood_probabilities[feature][class_label] = df[df['PlayTennis'] == class_label][feature].value_counts(normalize=True).to_dict()
        # Round the likelihood probabilities to 2 decimal places
        likelihood_probabilities[feature][class_label] = {k: round(v, 2) for k, v in likelihood_probabilities[feature][class_label].items()}
        print(f"Likelihood Probabilities for {feature} given {class_label}: ", likelihood_probabilities[feature][class_label])

Likelihood Probabilities for Outlook given No:  {'Sunny': 0.6, 'Rain': 0.4}
Likelihood Probabilities for Outlook given Yes:  {'Overcast': 0.44, 'Rain': 0.33, 'Sunny': 0.22}
Likelihood Probabilities for Temperature given No:  {'Hot': 0.4, 'Mild': 0.4, 'Cool': 0.2}
Likelihood Probabilities for Temperature given Yes:  {'Mild': 0.44, 'Cool': 0.33, 'Hot': 0.22}
Likelihood Probabilities for Humidity given No:  {'High': 0.8, 'Normal': 0.2}
Likelihood Probabilities for Humidity given Yes:  {'Normal': 0.67, 'High': 0.33}
Likelihood Probabilities for Wind given No:  {'Strong': 0.6, 'Weak': 0.4}
Likelihood Probabilities for Wind given Yes:  {'Weak': 0.67, 'Strong': 0.33}


In [7]:
# Navie Bayes Probability  for playTennis = Yes
prob_yes = prior_probabilities['Yes']
for feature, value in test_case.items():
    prob_yes *= likelihood_probabilities[feature]['Yes'].get(value, 0)

#print the probability of PlayTennis = Yes with two decimal places

print("Probability of PlayTennis = Yes: ", round(prob_yes,4))

Probability of PlayTennis = Yes:  0.0051


In [8]:
# Navie Bayes Probability  for playTennis = No
prob_no = prior_probabilities['No']
for feature, value in test_case.items():
    prob_no *= likelihood_probabilities[feature]['No'].get(value, 0)

#print the probability of PlayTennis = Yes with two decimal places

print("Probability of PlayTennis = No: ", round(prob_no,4))

Probability of PlayTennis = No:  0.0207


In [9]:
# The end result of NB is to compare the two probabilities and choose the one with the highest value.
if prob_yes > prob_no:
    print("The predicted class is: PlayTennis = Yes")
else:
    print("The predicted class is: PlayTennis = No")

The predicted class is: PlayTennis = No


In [10]:
# Lets implement the same code using sklearn library
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load the dataset
df = pd.read_csv('PlayTennis_NB_Example.csv')
df = df.drop(columns=['Day']) # Drop the 'Day' column as it is not needed for classification

# Encode categorical features using LabelEncoder
label_encoders = {}
for column in df.columns:
    if df[column].dtype == 'object':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le
        # Display the mapping for the column without showing int64()
        mapping = {original: int(encoded) for original, encoded in zip(le.classes_, le.transform(le.classes_))}
        print(f"Label Encoding for {column}: {mapping}")



# Split the dataset into features and target variable
X = df.drop(columns=['PlayTennis'])
y = df['PlayTennis']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Gaussian Naive Bayes model
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gnb.predict(X_test)

# Calculate accuracy and print classification report
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy in %: ", round(accuracy,2)*100)

Label Encoding for Outlook: {'Overcast': 0, 'Rain': 1, 'Sunny': 2}
Label Encoding for Temperature: {'Cool': 0, 'Hot': 1, 'Mild': 2}
Label Encoding for Humidity: {'High': 0, 'Normal': 1}
Label Encoding for Wind: {'Strong': 0, 'Weak': 1}
Label Encoding for PlayTennis: {'No': 0, 'Yes': 1}
Accuracy in %:  100.0


In [11]:
# Encode the test case with feature names
test_case_encoded = pd.DataFrame({
    'Outlook': label_encoders['Outlook'].transform([test_case['Outlook']])[0],
    'Temperature': label_encoders['Temperature'].transform([test_case['Temperature']])[0],
    'Humidity': label_encoders['Humidity'].transform([test_case['Humidity']])[0],
    'Wind': label_encoders['Wind'].transform([test_case['Wind']])[0]
}, index=[0])  # Create a DataFrame with feature names as columns

In [12]:
# Display the encoded test case with feature names
print("Encoded Test Case with Feature Names:")
print(test_case_encoded)

Encoded Test Case with Feature Names:
   Outlook  Temperature  Humidity  Wind
0        2            0         0     0


In [13]:
# display after inverse the encoding
decoded_test_case = test_case_encoded.copy()
for column in decoded_test_case.columns:
    decoded_test_case[column] = label_encoders[column].inverse_transform(decoded_test_case[column])
print("Given Test Case:")
print(decoded_test_case)

Given Test Case:
  Outlook Temperature Humidity    Wind
0   Sunny        Cool     High  Strong


In [14]:
# Make prediction for the test case
predicted_class = gnb.predict(test_case_encoded)[0]
predicted_class_label = label_encoders['PlayTennis'].inverse_transform([predicted_class])[0]
print("Predicted class for the test case:", predicted_class_label)

Predicted class for the test case: No


In [15]:
import pandas as pd

# Step 1: Create the dataset
data = {
    'CGPA': ['≥9', '≥8', '≥9', '<8', '≥8', '≥9', '<8', '≥9', '≥8', '≥8'],
    'Interactiveness': ['Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes'],
    'Practical Knowledge': ['Very good', 'Good', 'Average', 'Average', 'Good', 'Good', 'Good', 'Very good', 'Good', 'Average'],
    'Communication Skills': ['Good', 'Moderate', 'Poor', 'Good', 'Moderate', 'Moderate', 'Poor', 'Good', 'Good', 'Good'],
    'Job Offer': ['Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes']
}

df = pd.DataFrame(data)

# Step 2: Test case
test_case = {
    'CGPA': '≥9',
    'Interactiveness': 'Yes',
    'Practical Knowledge': 'Very good',
    'Communication Skills': 'Good'
}

# Step 3: Calculate prior probabilities
prior = df['Job Offer'].value_counts(normalize=True).to_dict()
print("Prior Probabilities:")
print(prior)

# Step 4: Calculate likelihoods
features = df.columns[:-1]
labels = df['Job Offer'].unique()
likelihoods = {}

for feature in features:
    likelihoods[feature] = {}
    for label in labels:
        subset = df[df['Job Offer'] == label]
        probs = subset[feature].value_counts(normalize=True).to_dict()
        likelihoods[feature][label] = probs

# Step 5: Apply Naive Bayes
results = {}
for label in labels:
    prob = prior[label]
    for feature in features:
        value = test_case[feature]
        prob *= likelihoods[feature][label].get(value, 0)
    results[label] = prob

# Step 6: Normalize (Optional)
total = sum(results.values())
for label in results:
    results[label] = round(results[label] / total, 4) if total > 0 else 0

print("\nPosterior Probabilities for Test Case:")
for k, v in results.items():
    print(f"{k}: {v}")

# Prediction
predicted = max(results, key=results.get)
print(f"\n✅ Predicted Job Offer: {predicted}")


Prior Probabilities:
{'Yes': 0.7, 'No': 0.3}

Posterior Probabilities for Test Case:
Yes: 1.0
No: 0.0

✅ Predicted Job Offer: Yes


In [16]:
from collections import defaultdict

# Dataset
data = [
    {"CGPA": "≥9", "Interactiveness": "Yes", "Practical": "Very good", "Communication": "Good", "Job": "Yes"},
    {"CGPA": "≥8", "Interactiveness": "No", "Practical": "Good", "Communication": "Moderate", "Job": "Yes"},
    {"CGPA": "≥9", "Interactiveness": "No", "Practical": "Average", "Communication": "Poor", "Job": "No"},
    {"CGPA": "<8", "Interactiveness": "No", "Practical": "Average", "Communication": "Good", "Job": "No"},
    {"CGPA": "≥8", "Interactiveness": "Yes", "Practical": "Good", "Communication": "Moderate", "Job": "Yes"},
    {"CGPA": "≥9", "Interactiveness": "Yes", "Practical": "Good", "Communication": "Moderate", "Job": "Yes"},
    {"CGPA": "<8", "Interactiveness": "Yes", "Practical": "Good", "Communication": "Poor", "Job": "No"},
    {"CGPA": "≥9", "Interactiveness": "No", "Practical": "Very good", "Communication": "Good", "Job": "Yes"},
    {"CGPA": "≥8", "Interactiveness": "Yes", "Practical": "Good", "Communication": "Good", "Job": "Yes"},
    {"CGPA": "≥8", "Interactiveness": "Yes", "Practical": "Average", "Communication": "Good", "Job": "Yes"},
]

# Test case
test_case = {"CGPA": "≥8", "Interactiveness": "Yes", "Practical": "Average", "Communication": "Good"}

# Helper: Get all possible values for each attribute
def get_attribute_values(data, attributes):
    values = defaultdict(set)
    for row in data:
        for attr in attributes:
            values[attr].add(row[attr])
    return values

# Step 1: Count priors and conditional probabilities
def train_naive_bayes(data, attributes, target):
    label_counts = defaultdict(int)
    conditional_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

    for row in data:
        label = row[target]
        label_counts[label] += 1
        for attr in attributes:
            conditional_counts[attr][row[attr]][label] += 1

    return label_counts, conditional_counts

# Step 2: Predict with Laplace smoothing
def predict(test_case, label_counts, conditional_counts, attr_values):
    total_instances = sum(label_counts.values())
    results = {}
    k = 1  # Laplace smoothing

    for label in label_counts:
        # Prior probability
        prob = label_counts[label] / total_instances

        for attr, value in test_case.items():
            num = conditional_counts[attr][value][label] + k
            denom = label_counts[label] + len(attr_values[attr])
            prob *= num / denom
        results[label] = prob

    # Normalize
    total_prob = sum(results.values())
    for label in results:
        results[label] = results[label] / total_prob

    return results

# Attributes and target
attributes = ["CGPA", "Interactiveness", "Practical", "Communication"]
target = "Job"

# Train
attr_values = get_attribute_values(data, attributes)
label_counts, conditional_counts = train_naive_bayes(data, attributes, target)

# Predict
posterior = predict(test_case, label_counts, conditional_counts, attr_values)

# Output
print("Posterior Probabilities:")
for label, prob in posterior.items():
    print(f"P({label} | test case) = {prob:.4f}")

print("\nPrediction:", max(posterior, key=posterior.get))


Posterior Probabilities:
P(Yes | test case) = 0.8750
P(No | test case) = 0.1250

Prediction: Yes
